In [1]:
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_config('configs/local_outcome_regression.yaml', max_workers=2)

tasks = benchmark.get_tasks()
tasks

In [5]:
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader
from sklearn.ensemble import RandomForestRegressor
from ppm_benchmark.utils.label_encoder import PPMLabelEncoder
from tqdm.notebook import tqdm

loader = BenchmarkLoader()
benchmark = loader.load_from_folder('outcome_regression')
tasks = benchmark.get_tasks()
results = dict()

for task_name in tqdm(tasks):
    task = benchmark.load_task(task_name)
    train = task.get_train_data()
    test = task.get_test_data().drop('target', axis=1)
    
    string_cols = train.select_dtypes(include=['object']).columns
    encoder = PPMLabelEncoder()
    train[string_cols] = encoder.fit_transform(train[string_cols])
    string_cols = test.select_dtypes(include=['object']).columns
    test[string_cols] = encoder.transform_with_new_labels(test[string_cols])
    
    X = train.drop('target', axis=1)
    y = train['target']
    
    model = RandomForestRegressor()
    model.fit(X, y)
        
    preds = model.predict(test)
    results[task.name] = preds

In [6]:
import pickle

with open('outcome_regression/test_results.pkl', 'wb') as f:
    pickle.dump(results, f)

## VANAF HIERONDER VOOR TEST.

In [2]:
import pickle
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_folder('outcome_regression')
evaluator = benchmark.get_evaluator()

with open('outcome_regression/test_results.pkl', 'rb') as f:
    results = pickle.load(f)

In [3]:
for task_name, result in results.items():
    task = benchmark.load_task(task_name)
    test = task.get_test_data().drop('target', axis=1)
    evaluator.add_predictions(task_name, result)
    

In [4]:
evaluator.evaluate()

In [5]:
evaluator.plot_by_fraction_completed('MAE')

In [6]:
evaluator.plot_by_train_act_distance('MAE')

In [7]:
evaluator.plot_by_attr_drift_column('MAE')